In [ ]:
orig_data = pd.read_csv(data_directory + 'qaData.csv', parse_dates=['Date'])
orig_data['EarningTag2'] = orig_data['EarningTag2'].str.strip()

#Add Year and Month, Quarter from Data
orig_data['Year'] = orig_data['Date'].dt.year
orig_data['Month'] = orig_data['Date'].dt.month
orig_data['Quarter'] = orig_data['Month'].apply(lambda x: 1 if x < 4 else 2 if x < 7 else 3 if x < 9 else 4)
orig_data['Company'] = orig_data['Company'].str.title().str.replace(" ", "")
orig_data['EventType'] = orig_data['EventType'].str.title().str.replace(" ", "")
orig_data['Participants'] = orig_data['Participants'].str.title().str.replace(" ", "")
orig_data['AnalystName'] = orig_data['AnalystName'].str.title().str.replace(" ", "")
orig_data['AnalystCompany'] = orig_data['AnalystCompany'].str.title().str.replace(" ", "")
orig_data['Tag'] = orig_data['EarningTag2'].str.title().str.replace(" ", "")

orig_data = orig_data.loc[~orig_data['AnalystName'].isna()].copy()

tag_cols = orig_data['Tag'].unique().tolist()

#Index Data
groups = []
for i, (name, group) in enumerate(orig_data.groupby(['Company', 'Participants', 'Month', 'Year', 'Quarter', 'EventType', 'Date'])):
    g2 = group.copy()
    g2['EventNumber'] = i
    groups.append(g2)
    
indexed_data = pd.concat(groups)

analyst_data = pd.read_csv(data_directory+"analystTopic.csv")
a_topic_cols = analyst_data.drop(['AnalystName', 'EventNumber', 'a_tMax'], axis=1).columns.tolist()
topic_cols = a_topic_cols.copy()
indexed_data = indexed_data.merge(analyst_data[['AnalystName', 'EventNumber', 'a_tMax']], on=['AnalystName','EventNumber'])

tag_data = pd.read_csv(data_directory+"tagTopic.csv")
t_topic_cols = tag_data.drop(['Tag', 'EventNumber', 't_tMax'], axis=1).columns.tolist()
topic_cols += t_topic_cols.copy()
indexed_data = indexed_data.merge(tag_data[['Tag', 'EventNumber', 't_tMax']], on=['Tag','EventNumber'])

cols_save = ['EventNumber','Company', 'Month', 'Year', 'Quarter', 'EventType']

pivot_data = pd.pivot_table(indexed_data, index=cols_save, columns=['a_tMax','t_tMax'], aggfunc='size', fill_value=0).reset_index()
pivot_data.columns = ['__'.join(col).rstrip('__') for col in pivot_data.columns.values]
melt_data = pd.melt(pivot_data, id_vars=cols_save, var_name=['a_tMax__t_tMax'], value_name='NumQ')
melt_data[['a_tMax', 't_tMax']] = melt_data['a_tMax__t_tMax'].str.split("__", expand=True)
melt_data['NumQ'] = melt_data['NumQ'].astype(bool).astype(int)
melt_data = pd.concat([melt_data,
                          pd.get_dummies(melt_data[['a_tMax', 't_tMax', 'Company', 'EventType']])], axis=1).reset_index(drop=True)

features_data = melt_data.drop(['Company', 'EventType', 'a_tMax', 't_tMax', 'a_tMax__t_tMax'], axis=1).copy()
train, test = features_data.loc[~features_data['EventNumber'].isin(test_set)].copy().reset_index(drop=True), \
                features_data.loc[features_data['EventNumber'].isin(test_set)].copy().reset_index(drop=True)

X_train, y_train = train.drop(['NumQ','EventNumber'], axis=1).values, train['NumQ'].values
X_test, y_test = test.drop(['NumQ', 'EventNumber'], axis=1).values, test['NumQ'].values

cols_list = train.drop(['NumQ','EventNumber'], axis=1).columns.values

print("Data Size: {}".format(features_data.shape))

estimator = RandomForestClassifier(warm_start=True, n_estimators=1000).fit(X_train, y_train)
preds = estimator.predict_proba(X_test)[:,1]
print(roc_auc_score(y_test, preds))
print(accuracy_score(y_test, preds.round()))

In [ ]:
orig_data = pd.read_csv(data_directory + 'qaData.csv', parse_dates=['Date'])
orig_data['EarningTag2'] = orig_data['EarningTag2'].str.strip()

#Add Year and Month, Quarter from Data
orig_data['Year'] = orig_data['Date'].dt.year
orig_data['Month'] = orig_data['Date'].dt.month
orig_data['Quarter'] = orig_data['Month'].apply(lambda x: 1 if x < 4 else 2 if x < 7 else 3 if x < 9 else 4)
orig_data['Company'] = orig_data['Company'].str.title().str.replace(" ", "")
orig_data['EventType'] = orig_data['EventType'].str.title().str.replace(" ", "")
orig_data['Participants'] = orig_data['Participants'].str.title().str.replace(" ", "")
orig_data['AnalystName'] = orig_data['AnalystName'].str.title().str.replace(" ", "")
orig_data['AnalystCompany'] = orig_data['AnalystCompany'].str.title().str.replace(" ", "")
orig_data['Tag'] = orig_data['EarningTag2'].str.title().str.replace(" ", "")

orig_data = orig_data.loc[~orig_data['AnalystName'].isna()].copy()

tag_cols = orig_data['Tag'].unique().tolist()

#Index Data
groups = []
for i, (name, group) in enumerate(orig_data.groupby(['Company', 'Participants', 'Month', 'Year', 'Quarter', 'EventType', 'Date'])):
    g2 = group.copy()
    g2['EventNumber'] = i
    groups.append(g2)
    
indexed_data = pd.concat(groups)

analyst_data = pd.read_csv(data_directory+"analystTopic.csv")
a_topic_cols = analyst_data.drop(['AnalystName', 'EventNumber'], axis=1).columns.tolist()
topic_cols = a_topic_cols.copy()
indexed_data = indexed_data.merge(analyst_data, on=['AnalystName','EventNumber'])

tag_data = pd.read_csv(data_directory+"tagTopic.csv")
t_topic_cols = tag_data.drop(['Tag', 'EventNumber'], axis=1).columns.tolist()
topic_cols += t_topic_cols.copy()
indexed_data = indexed_data.merge(tag_data, on=['Tag','EventNumber'])





cols_save = ['EventNumber','Company', 'Month', 'Year', 'Quarter', 'EventType', 'Date']

pivot_data = pd.pivot_table(indexed_data, index=cols_save, columns=['a_tMax', 'Tag'], aggfunc='size', fill_value=0).reset_index()
pivot_data.columns = ['__'.join(col).rstrip('__') for col in pivot_data.columns.values]
melt_data = pd.melt(pivot_data, id_vars=cols_save, var_name=['a_tMax__Tag'], value_name='NumQ')
melt_data[['a_tMax', 'Tag']] = melt_data['a_tMax__Tag'].str.split("__", expand=True)
melt_data['NumQ'] = melt_data['NumQ'].astype(bool).astype(int)
melt_data = pd.concat([melt_data, 
                        pd.get_dummies(melt_data[['Company', 'EventType', 'Tag']])], axis=1).reset_index(drop=True)

#for i in ['a_tMax', 'Company', 'Quarter']:
    
#    hist_data = historicVarPct(melt_data, i, 'Tag')
#    melt_data = pd.merge(melt_data, hist_data, on=[i, 'Company', 'Date'])


features_data = melt_data.drop(['Company', 'Tag', 'EventType', 'Date' , 'Year', 'a_tMax', 'a_tMax__Tag'], axis=1).copy()
features_data = features_data.reset_index(drop=True)
features_data['NumQ'] = features_data['NumQ'].astype(bool).astype(int)

train, test = features_data.loc[~features_data['EventNumber'].isin(test_set)].copy().reset_index(drop=True), \
                features_data.loc[features_data['EventNumber'].isin(test_set)].copy().reset_index(drop=True)

X_train, y_train = train.drop(['NumQ','EventNumber'], axis=1).values, train['NumQ'].values
X_test, y_test = test.drop(['NumQ', 'EventNumber'], axis=1).values, test['NumQ'].values

cols_list = train.drop(['NumQ','EventNumber'], axis=1).columns.values